In [22]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import yaml, psycopg2
from psycopg2.extensions import AsIs

# Connect to Postgres
with open('./credentials', 'r') as credential_yaml:
    credentials = yaml.load(credential_yaml)

with open('./config', 'r') as config_yaml:
    config = yaml.load(config_yaml)

    
snorkel_connection = psycopg2.connect(
    dbname=credentials['snorkel_postgres']['database'],
    user=credentials['snorkel_postgres']['user'],
    password=credentials['snorkel_postgres']['password'],
    host=credentials['snorkel_postgres']['host'],
    port=credentials['snorkel_postgres']['port'])
snorkel_cursor = snorkel_connection.cursor()

In [2]:

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS candidate CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS context CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS document CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS feature CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS feature_key CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS gold_label CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS gold_label_key CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS label CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS label_key CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS marginal CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS prediction CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS prediction_key CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS sentence CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS span CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS spouse CASCADE;
""")

snorkel_cursor.execute("""
    DROP TABLE IF EXISTS stable_label CASCADE;
""")

snorkel_connection.commit()


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
###
os.environ['SNORKELDB'] = 'postgres://BKumpf@localhost:5432/snorkel'

###
from snorkel import SnorkelSession
session = SnorkelSession()


In [4]:
# Connect to Postgres
with open('./credentials', 'r') as credential_yaml:
    credentials = yaml.load(credential_yaml)

with open('./config', 'r') as config_yaml:
    config = yaml.load(config_yaml)

# Connect to Postgres
connection = psycopg2.connect(
    dbname=credentials['postgres']['database'],
    user=credentials['postgres']['user'],
    password=credentials['postgres']['password'],
    host=credentials['postgres']['host'],
    port=credentials['postgres']['port'])
cursor = connection.cursor()

snorkel_connection = psycopg2.connect(
    dbname=credentials['snorkel_postgres']['database'],
    user=credentials['snorkel_postgres']['user'],
    password=credentials['snorkel_postgres']['password'],
    host=credentials['snorkel_postgres']['host'],
    port=credentials['snorkel_postgres']['port'])
snorkel_cursor = snorkel_connection.cursor()

cursor.execute("""
    SELECT DISTINCT(docid) FROM evap_sentences_nlp352;
"""
)

count = 1
for docid in cursor:
    snorkel_cursor.execute("INSERT INTO context (id, type, stable_id) VALUES (nextval('context_id_seq'), 'document', %(stable_id)s)", {"stable_id": docid[0] + "::document:0:0"})
    snorkel_cursor.execute("INSERT INTO document (id, name) VALUES (currval('context_id_seq'), %(docid)s)", {"count" : count, "docid": docid[0]})
    snorkel_connection.commit()
    count += 1

#IMPORT THE SENTENCES DUMP
cursor.execute("""
            SELECT docid, sentid, words, poses, ners, lemmas, dep_paths, dep_parents
            FROM %(my_app)s_sentences_%(my_product)s ORDER BY docid, sentid;
            """, {
                "my_app": AsIs(config['app_name']),
                    "my_product": AsIs(config['product'].lower())
                    })

# Need to get document-level offsets for stable_id at the sentence level.
count = 1

doc_char_counts = {}

for sent in cursor:
    parsed_sent = {}
    snorkel_cursor.execute("SELECT id FROM document WHERE name=%(docid)s", {"docid" : sent[0]})
    document_id = snorkel_cursor.fetchone()[0]
    parsed_sent["document_id"] = document_id
    parsed_sent["position"] = sent[1]
    parsed_sent["words"] = sent[2]
    parsed_sent["pos_tags"] = sent[3]
    parsed_sent["ner_tags"] = sent[4]
    parsed_sent["lemmas"] = sent[5]
    parsed_sent["dep_labels"] = sent[6]
    parsed_sent["dep_parents"] = sent[7]
    parsed_sent["text"] = " ".join(word for word in parsed_sent["words"])
    parsed_sent["char_offsets"] = [0 for i in range(len(parsed_sent["words"]))]
    parsed_sent["abs_char_offsets"] = [0 for i in range (len(parsed_sent["words"]))]
    parsed_sent["entitiy_cids"] = ['O']

        
    sentence_running_count = 0
    for wordidx in range(len(parsed_sent["words"])):
        parsed_sent["char_offsets"][wordidx] = sentence_running_count
        sentence_running_count += len(parsed_sent["words"][wordidx]) + 1

    # This will probably be off by one...
    if sent[0] in doc_char_counts:
        sentence_start = doc_char_counts[sent[0]] + 1
        doc_char_counts[sent[0]] += sentence_running_count
    else:
        sentence_start = 0
        doc_char_counts[sent[0]] = sentence_running_count

    # keep this running count as the sentence-level offset stable_id
    snorkel_cursor.execute("INSERT INTO context (id, type, stable_id) VALUES (nextval('context_id_seq'), 'sentence', %(stable_id)s)", {"stable_id": sent[0] + "::sentence:%s:%s" % (sentence_start, doc_char_counts[sent[0]])})

    snorkel_connection.commit()
    snorkel_cursor.execute(" \
        INSERT INTO sentence (id, document_id, position, words, pos_tags, ner_tags, lemmas, dep_labels, dep_parents, char_offsets, abs_char_offsets, text) VALUES \
                (currval('context_id_seq'), \
                %(document_id)s, \
                %(position)s, \
                %(words)s, \
                %(pos_tags)s, \
                %(ner_tags)s,  \
                %(lemmas)s, \
                %(dep_labels)s, \
                %(dep_parents)s, \
                %(char_offsets)s, \
                %(abs_char_offsets)s, \
                %(text)s);", parsed_sent)
    snorkel_connection.commit()
    count += 1

snorkel_cursor.close()
snorkel_connection.close()
cursor.close()
connection.close()


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [5]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

('Documents:', 296L)
('Sentences:', 206435L)


## Defining a Candidate schema

#### We now define the schema of the relation mention we want to extract (which is also the schema of the candidates). This must be a subclass of Candidate, and we define it using a helper function. Here we'll define a binary spouse relation mention which connects two Span objects of text. Note that this function will create the table in the database backend if it does not exist:

## Evaporite schema

* By using the `canidate schema` we will map the three things we need snorkel to extract, `Stratigraphic description`, `Mineral Information`, and a third function to attempt and extract the `age` from within the document. 

* `Strat` subclass has three relations, the `name` of the unit, unit `classification` (Super Group, Group, Formation, etc..) and the `target_word` it was matched too. 

* `Strat age` will be collected independentally of identifying the stratigraphic unit... Could use macrostrat to fill in the age of a given unit. 

* `Mineral` matches a target word and a mineral word and fills the span between them. 




In [6]:
target_words = ('evaporite','potash','soda lake', 'salt deposit')
target_wordss = ('evaporite','potash','soda lake', 'salt deposit')
target_dict = dict(zip(target_words,target_wordss))

In [7]:
mineral_words = {'halite', 'sylvite','hydrohalite','bischofite', 'antacitite','carnallite','tachyhydrite','rhinneite','kainite','aragonite','calcite','thermonatrite','soda','nahcolite','dolomite','anhydrite','bassanite', 'gypsum' , 'kieserite', 'saderite', 'epsomite', 'thernadite', 'mirabillte', 'celestine', 'glaserite', 'vanthoffite', 'bleodite', 'loeweite', 'langbeinite' , 'leonine', 'glauberite', 'syngenite', 'polyhalite', 'northupite', 'burette', 'techie', 'hanksite', 'lautarite', 'kernite', 'borax', 'colemanite' }              
mineral_wordss = {'halite', 'sylvite','hydrohalite','bischofite', 'antacitite','carnallite','tachyhydrite','rhinneite','kainite','aragonite','calcite','thermonatrite','soda','nahcolite','dolomite','anhydrite','bassanite', 'gypsum' , 'kieserite', 'saderite', 'epsomite', 'thernadite', 'mirabillte', 'celestine', 'glaserite', 'vanthoffite', 'bleodite', 'loeweite', 'langbeinite' , 'leonine', 'glauberite', 'syngenite', 'polyhalite', 'northupite', 'burette', 'techie', 'hanksite', 'lautarite', 'kernite', 'borax', 'colemanite' }              
mineral_dict = dict(zip(mineral_words,mineral_wordss))



In [21]:
## Copied From Jon's Stromatolite Demo 'strat_var'
#words indicating stratigraphic names
#strat_flags = ["Group", "Formation", "Member", "Supergroup", "Bed", "Subgroup","Gp.", "Fm.", "Mbr.", "SGp.", "Gp", "Fm", "Mbr", "SGp"]
    
#lith_flags = ["Dolomite","Dolostone","Limestone","Sandstone","Shale","Conglomerate","Chert"]

#strat_flags = strat_flags+lith_flags
#strat_flagss = strat_flags
#strat_dict = dict(zip(strat_flags,strat_flagss))
#print strat_dict              

#words indicating an age
#age_flags = ["Ma.", "Ga.", "Myr.","Ma", "Ga", "Myr"]

#list of known and troublesome ligatures
#weird_strings = [['\xef\xac\x82', 'fl'], ['\xef\xac\x81', 'fi']]

{'Fm.': 'Fm.', 'Gp.': 'Gp.', 'Limestone': 'Limestone', 'Group': 'Group', 'Gp': 'Gp', 'Sandstone': 'Sandstone', 'SGp': 'SGp', 'Supergroup': 'Supergroup', 'Bed': 'Bed', 'Fm': 'Fm', 'Member': 'Member', 'Chert': 'Chert', 'Shale': 'Shale', 'Formation': 'Formation', 'Dolostone': 'Dolostone', 'Dolomite': 'Dolomite', 'SGp.': 'SGp.', 'Conglomerate': 'Conglomerate', 'Mbr.': 'Mbr.', 'Mbr': 'Mbr', 'Subgroup': 'Subgroup'}


In [8]:
from snorkel.models import candidate_subclass

#Strat = candidate_subclass('Strat', ['strat_name', 'strat_unit','target_word'])
#Strat_Age = canidate_subclass('Strat_Age',['strat_name','strat_age'])
Mineral = candidate_subclass('Mineral' ,['mineral_word','target word'])


In [9]:
# ngrams are words considered for a given sentence. 

from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DictionaryMatch

ngrams         = Ngrams(n_max=20)
mineral_matcher = DictionaryMatch(ignore_case=True, d=mineral_dict)
target_matcher  = DictionaryMatch(ignore_case=True, d=target_dict)
cand_extractor = CandidateExtractor(Mineral, [ngrams, ngrams], [mineral_matcher, target_matcher])


In [11]:
from snorkel.models import Document
docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for x,s in enumerate(doc.sentences):
        if  x  <= 68811:
            dev_sents.add(s)
        elif  x <= 137622 and i > 68811:
            test_sents.add(s)
        else:
            train_sents.add(s)

In [12]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Mineral).filter(Mineral.split == i).count())

Clearing existing...


0it [00:00, ?it/s]
  0%|          | 0/206435 [00:00<?, ?it/s]

Running UDF...
('Number of candidates:', 0L)
Clearing existing...
Running UDF...


100%|██████████| 206435/206435 [30:03<00:00, 114.44it/s] 
0it [00:00, ?it/s]

('Number of candidates:', 32L)
Clearing existing...
Running UDF...
('Number of candidates:', 0L)
CPU times: user 27min, sys: 1min 57s, total: 28min 57s
Wall time: 30min 5s
